In [ ]:
import csv
import cv2
import numpy as np
import librosa
import librosa.display
import wave
import os
import matplotlib.pyplot as plt
import multiprocessing 
import pickle


In [ ]:
SOURCE_FOLDER_TRAIN= '../AudioData/audio-cats-and-dogs/cats_dogs/train/'
OUTPUT_FOLDER_TRAIN = '../GeneratedData/audio-cats-and-dogs/cats_dogs/train/'

In [ ]:
SOURCE_FOLDER_TEST= '../AudioData/audio-cats-and-dogs/cats_dogs/test/'
OUTPUT_FOLDER_TEST = '../GeneratedData/audio-cats-and-dogs/cats_dogs/test/'

In [9]:
FileName =input("Type Filename befor the '.' => ")
SpectrumVariables = pickle.load(open( FileName+'.SpectrumVariables', "rb" ) )
SpectrumVariables

Type Filename befor the '.' => test6


{'RESOLUTION': 224,
 'SAMPLE_RATE': 48000,
 'N_FFT': 2048,
 'HOP_LENGTH': 265,
 'FMIN': 60,
 'FMAX': 15000,
 'N_MELS': 200,
 'POWER': 2.0}

In [10]:
def log_mel_spec_tfm(src_path, dst_path):
    print(src_path)
    fname=src_path.split('/')[-1];
    x=[]
    sample_rate=0
    try:
        channels ,sample_rate_in = librosa.load(src_path,mono=False)
    except:
        print(fname+" Could not be computed!")
        return
    if(channels.ndim==1):
        channels= channels.reshape((1,(len(channels))))
    channelcounter=0
    for x in channels:
        channelcounter+=1
        
        RESOLUTION=SpectrumVariables["RESOLUTION"] 
        SAMPLE_RATE=SpectrumVariables["SAMPLE_RATE"]
        N_FFT=SpectrumVariables["N_FFT"]
        HOP_LENGTH= SpectrumVariables["HOP_LENGTH"]
        FMIN=SpectrumVariables["FMIN"]
        FMAX=SpectrumVariables["FMAX"]
        N_MELS=SpectrumVariables["N_MELS"]
        POWER=SpectrumVariables["POWER"]
        audio_data = librosa.resample(x, sample_rate_in, SAMPLE_RATE)
        mel_spec_power = librosa.feature.melspectrogram(audio_data, sr=SAMPLE_RATE, n_fft=N_FFT, 
                                                        hop_length=HOP_LENGTH, 
                                                        n_mels=N_MELS, power=POWER,
                                                       fmin=FMIN,fmax=FMAX)
        mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max)
        image = mel_spec_db; # convert to float
        image -= image.min() # ensure the minimal value is 0.0
        image /= image.max() # maximum value in image is now 1.0
        image*=256;
        img = image.astype(np.uint8)
        colerPic = cv2.applyColorMap(img, cv2.COLORMAP_BONE)
        count=0
        for i in range(int(np.floor(colerPic.shape[1]/RESOLUTION))):
            count+=1
            startx=RESOLUTION*i
            stopx=RESOLUTION*(i+1)
            OutputImage = cv2.resize(colerPic[:,startx:stopx,:],(RESOLUTION,RESOLUTION))
            plt.imsave(os.path.join(dst_path,(fname.replace(".wav",'-').replace(".m4a",'-').replace(".mp3",'-') +str(i)+'-ch-'+str(channelcounter)+ '.png')), OutputImage) 
        if(count==0):
            print("WARNING: => Jumped A file. If you see this often, increas sampleRate or Hop length.")

In [11]:
def ComputeSpectrograms(IN_FOLDER,OUT_FOLDER):
    SourceFoldersLabels = [f.path for f in os.scandir(IN_FOLDER) if f.is_dir()]
    for path in SourceFoldersLabels:
        FileList = [f.path for f in os.scandir(path) if f.is_file() and (f.name.endswith(".wav") or f.name.endswith(".m4a") or f.name.endswith(".mp3"))]
        Label = path.split('/')[-1]
        outFolder = os.path.join(OUT_FOLDER,Label)
        if not os.path.exists(outFolder):
            os.makedirs(outFolder)
        with multiprocessing.Pool(4) as p:
            p.starmap(log_mel_spec_tfm, [(f,outFolder) for f in FileList])
        print("Finished this pool. Now next Folder!")
    print("Done generating spectra!")  

In [8]:

ComputeSpectrograms(SOURCE_FOLDER_TRAIN,OUTPUT_FOLDER_TRAIN)
ComputeSpectrograms(SOURCE_FOLDER_TEST,OUTPUT_FOLDER_TEST)

../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_126.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_88.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_124.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_134.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_130.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_121.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_99.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_135.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_109.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_131.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_132.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_98.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_119.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_108.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_151.wav
../AudioData/audio-cats-and-

../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_21.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_0.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_6.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_23.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_30.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_26.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_4.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_5.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_20.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_37.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_18.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_33.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_36.wav
../AudioData/audio-cats-and-dogs/cats_dogs/train/dog/dog_barking_31.

../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/cat_155.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/cat_15.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/cat_141.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/cat_13.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/cat_39.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/cat_25.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/cat_21.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/cat_19.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/cat_33.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/cat_3.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/cat_83.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/cat_35.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/cat_27.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/cat_68.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/cat_149.wav
../AudioData/audio-cats-and-dogs/cats_dogs/test/cat/c

In [12]:
ComputeSpectrograms("../AudioData/urban-sound-classification/train/Train/","../GeneratedData/urban-sound-classification/train/Train/")

../AudioData/urban-sound-classification/train/Train/siren/2002.wav
../AudioData/urban-sound-classification/train/Train/siren/4100.wav
../AudioData/urban-sound-classification/train/Train/siren/850.wav
../AudioData/urban-sound-classification/train/Train/siren/7829.wav
../AudioData/urban-sound-classification/train/Train/siren/4473.wav
../AudioData/urban-sound-classification/train/Train/siren/2601.wav
../AudioData/urban-sound-classification/train/Train/siren/1294.wav
../AudioData/urban-sound-classification/train/Train/siren/8706.wav
../AudioData/urban-sound-classification/train/Train/siren/663.wav
../AudioData/urban-sound-classification/train/Train/siren/6270.wav
../AudioData/urban-sound-classification/train/Train/siren/728.wav
../AudioData/urban-sound-classification/train/Train/siren/249.wav
../AudioData/urban-sound-classification/train/Train/siren/922.wav
../AudioData/urban-sound-classification/train/Train/siren/2198.wav
../AudioData/urban-sound-classification/train/Train/siren/5637.wav


../AudioData/urban-sound-classification/train/Train/siren/6077.wav
../AudioData/urban-sound-classification/train/Train/siren/2212.wav
../AudioData/urban-sound-classification/train/Train/siren/923.wav
../AudioData/urban-sound-classification/train/Train/siren/1242.wav
../AudioData/urban-sound-classification/train/Train/siren/5218.wav
../AudioData/urban-sound-classification/train/Train/siren/3682.wav
../AudioData/urban-sound-classification/train/Train/siren/2206.wav
../AudioData/urban-sound-classification/train/Train/siren/3685.wav
../AudioData/urban-sound-classification/train/Train/siren/1068.wav
../AudioData/urban-sound-classification/train/Train/siren/7418.wav
../AudioData/urban-sound-classification/train/Train/siren/3696.wav
../AudioData/urban-sound-classification/train/Train/siren/4110.wav
../AudioData/urban-sound-classification/train/Train/siren/6931.wav
../AudioData/urban-sound-classification/train/Train/siren/1069.wav
../AudioData/urban-sound-classification/train/Train/siren/8270.

../AudioData/urban-sound-classification/train/Train/siren/3813.wav
../AudioData/urban-sound-classification/train/Train/siren/6548.wav
../AudioData/urban-sound-classification/train/Train/siren/3186.wav
../AudioData/urban-sound-classification/train/Train/siren/60.wav
../AudioData/urban-sound-classification/train/Train/siren/3191.wav
../AudioData/urban-sound-classification/train/Train/siren/4164.wav
../AudioData/urban-sound-classification/train/Train/siren/943.wav
../AudioData/urban-sound-classification/train/Train/siren/5095.wav
../AudioData/urban-sound-classification/train/Train/siren/4601.wav
../AudioData/urban-sound-classification/train/Train/siren/6767.wav
../AudioData/urban-sound-classification/train/Train/siren/5069.wav
../AudioData/urban-sound-classification/train/Train/siren/2936.wav
../AudioData/urban-sound-classification/train/Train/siren/6016.wav
../AudioData/urban-sound-classification/train/Train/siren/1746.wav
../AudioData/urban-sound-classification/train/Train/siren/1958.wa

../AudioData/urban-sound-classification/train/Train/siren/5936.wav
../AudioData/urban-sound-classification/train/Train/siren/4027.wav
../AudioData/urban-sound-classification/train/Train/siren/8187.wav
../AudioData/urban-sound-classification/train/Train/siren/427.wav
../AudioData/urban-sound-classification/train/Train/siren/5063.wav
../AudioData/urban-sound-classification/train/Train/siren/6804.wav
../AudioData/urban-sound-classification/train/Train/siren/6156.wav
../AudioData/urban-sound-classification/train/Train/siren/3978.wav
../AudioData/urban-sound-classification/train/Train/siren/8190.wav
../AudioData/urban-sound-classification/train/Train/siren/1831.wav
../AudioData/urban-sound-classification/train/Train/siren/8547.wav
../AudioData/urban-sound-classification/train/Train/siren/3944.wav
../AudioData/urban-sound-classification/train/Train/siren/6817.wav
../AudioData/urban-sound-classification/train/Train/siren/3798.wav
../AudioData/urban-sound-classification/train/Train/siren/6224.

../AudioData/urban-sound-classification/train/Train/siren/7016.wav
../AudioData/urban-sound-classification/train/Train/siren/2592.wav
../AudioData/urban-sound-classification/train/Train/siren/687.wav
../AudioData/urban-sound-classification/train/Train/siren/4523.wav
../AudioData/urban-sound-classification/train/Train/siren/2019.wav
../AudioData/urban-sound-classification/train/Train/siren/7367.wav
../AudioData/urban-sound-classification/train/Train/siren/5613.wav
../AudioData/urban-sound-classification/train/Train/siren/7980.wav
../AudioData/urban-sound-classification/train/Train/siren/912.wav
../AudioData/urban-sound-classification/train/Train/siren/4652.wav
../AudioData/urban-sound-classification/train/Train/siren/8445.wav
../AudioData/urban-sound-classification/train/Train/siren/5826.wav
../AudioData/urban-sound-classification/train/Train/siren/2794.wav
../AudioData/urban-sound-classification/train/Train/siren/5558.wav
../AudioData/urban-sound-classification/train/Train/siren/2815.w

...

../AudioData/urban-sound-classification/train/Train/jackhammer/3347.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/1768.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/4474.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/6007.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/6992.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/5736.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/7169.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/5268.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/7326.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/6773.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/3333.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/4162.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/599.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/41

../AudioData/urban-sound-classification/train/Train/jackhammer/6616.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/945.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/363.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/2714.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/3236.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/2312.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/1347.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/6629.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/4365.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/6606.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/3751.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/6372.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/402.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/6572

../AudioData/urban-sound-classification/train/Train/jackhammer/1176.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/4390.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/2452.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/6620.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/4197.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/4581.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/7869.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/140.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/4829.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/7936.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/740.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/5671.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/5504.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/357

../AudioData/urban-sound-classification/train/Train/jackhammer/7198.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/6483.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/178.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/730.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/4485.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/6118.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/2792.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/2697.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/8280.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/7560.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/2418.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/6457.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/1605.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/863

../AudioData/urban-sound-classification/train/Train/jackhammer/2142.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/6938.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/1473.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/4246.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/2624.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/269.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/3264.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/6442.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/655.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/4685.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/8127.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/7990.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/1058.wav
../AudioData/urban-sound-classification/train/Train/jackhammer/329

In [ ]:
'''
Unused function
'''
def generateOutputFolder(overwrite, path,labels_local):
    if not os.path.exists(path):
        #if(overwrite):
            #TODO actually delete the folder
        os.makedirs(path)
    TemporaryDict={}
    for i in labels_local:
        LabeldDirectory = os.path.join(path,i)
        if not os.path.exists(LabeldDirectory):
            os.makedirs(LabeldDirectory)
        TemporaryDict[i]=LabeldDirectory
    return TemporaryDict


